<a href="https://colab.research.google.com/github/pgallino/Organizacion-de-datos-TP1-1C2023/blob/main/spark4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=20460326e8649a77330390a1cf6793915e973f8a80820e1351d6766ace24a379
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Wikipedia dataset 2c2021/pagelinks_sample.csv', header=True, inferSchema=True)
rdd_pagelinks = df.rdd
df2 = sqlContext.read.csv('/content/drive/MyDrive/Wikipedia dataset 2c2021/contents.csv', header=True, inferSchema=True)
rdd_contents = df2.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


4 Considerando el pagelink_sample.csv representar como un grafo en Spark los contenidos de wikipedia (considerando los contenidos como nodos y los links como aristas) como una lista de aristas y mostrar un recorrido en la estructura. (⭐⭐⭐)

Como en pagelinks_sample me aparecen titulos de articulos sin id, utilizo contents.csv para mapear esos titulos al id correspondiente.

Por errores en los datos, filtro limpiando los que no tengan un id numérico.

In [ ]:
rdd_contents = rdd_contents.filter(lambda x: x.title != None and x.id != None and x.id.isdigit())

Me quedo con los títulos y los ids

In [ ]:
rdd_contents = rdd_contents.map(lambda x: (x.title, x.id))

In [ ]:
rdd_contents.take(5)

[('Wikipedia:Artículos solicitados', '5'),
 ('Andorra', '7'),
 ('Argentina', '10'),
 ('Geografía de Andorra', '15'),
 ('Demografía de Andorra', '17')]

In [ ]:
rdd_contents = rdd_contents.map(lambda x: (x[0], int(x[1]))).cache()

Lo doy vuelta para tener los ids asociados a títulos para imprimir luego las capas del bfs.

In [ ]:
rdd_keys = rdd_contents.map(lambda x: (x[1], x[0])).cache()

In [ ]:
rdd_keys.take(5)

[(5, 'Wikipedia:Artículos solicitados'),
 (7, 'Andorra'),
 (10, 'Argentina'),
 (15, 'Geografía de Andorra'),
 (17, 'Demografía de Andorra')]

Del csv de pagelinks, me quedo con las dos columnas que me interesan.

In [ ]:
rdd_pagelinks = rdd_pagelinks.map(lambda x: (x.pl_title, x.pl_from)).cache()

In [ ]:
rdd_pagelinks.take(5)

[('Bandera_cÃ³digo_del_COI_TPE', 4840492),
 ('Esteban_DuÅ¡an', 950948),
 ('Insecta', 6418187),
 ('Pleasure_to_Kill', 3436623),
 ('DamiÃ¡n_Oliver', 4713066)]

Para obtener las aristas, hago join entre los rdd.
En contents tengo (titulo, id)
En pagelinks tengo (titulo al que va el link, id donde está el link)
Quedandome registros de la forma (Titulo, (id, id donde está el link)).
Las aristas serian (id donde está el link, id) siendo id, el id del articulo al que va el link.

In [ ]:
rdd_aristas = rdd_pagelinks.join(rdd_contents)

Reformo los registros para que se conviertan en las aristas.

In [ ]:
rdd_aristas = rdd_aristas.map(lambda x: (x[1][1], x[1][0])).cache()

In [ ]:
rdd_aristas.take(5)

[(29123, 672647),
 (29123, 181873),
 (29123, 1057771),
 (29123, 9715401),
 (29123, 5100785)]

Hasta aca conseguí la representación del grafo por listado de aristas. Para realizar el recorrido mediante BFS, convierto el listado de aristas a lista de adyacencia.

Para ello simplemente hago groupByKey() obteniendo registros de la forma (id, listado de vecinos)

In [ ]:
ady = rdd_aristas.groupByKey()

In [ ]:
ady.take(5)

[(1075224, <pyspark.resultiterable.ResultIterable at 0x7fb86b094af0>),
 (5888616, <pyspark.resultiterable.ResultIterable at 0x7fb86b096aa0>),
 (160200, <pyspark.resultiterable.ResultIterable at 0x7fb86b095e40>),
 (1502256, <pyspark.resultiterable.ResultIterable at 0x7fb86b094ac0>),
 (1916808, <pyspark.resultiterable.ResultIterable at 0x7fb86b095cc0>)]

A partir de está linea, me baso en los ejemplos subidos por la cátedra sobre ejercicios de grafos/Kevin bacon para realizar el recorrido BFS. 
Ya no necesito la traza porque no estoy buscando caminos mínimos. En vez de arrancar por Kevin Bacon, tomo un nodo cualquiera. Además ya no calculo la cantidad de nodos de x grado, simplemente los imprimo.

Construyo registros de formato (node_id, (distance, status, [list of neighbors])

In [ ]:
adjlist = ady.map(lambda x: (x[0], (999, '-', x[1]))).cache()

Tomo las funciones proc_node y reduce_nodes de la práctica, quitando la parte donde se trabaja sobre la traza.

In [ ]:
def proc_node(node):
  if node[1][0]==999:
    new_distance = 1
  else:
    new_distance = node[1][0]+1
  ret = []
  for neighbor in node[1][2]:
    ret.append((neighbor,(new_distance,'P',[])))
  ret.append((node[0],(node[1][0],'DONE',node[1][2])))
  return ret

In [ ]:
def reduce_nodes(n1,n2):
  new_status = ''
  if n1[1]=='P' or n2[1]=='P':
    new_status = 'P'
  if n1[1]=='DONE' or n2[1]=='DONE':
    new_status = 'DONE'
  if n1[0]<n2[0]:
    new_distance = n1[0]
  else:
    new_distance = n2[0]
  new_list = n1[2]
  if len(n2[2])>len(n1[2]):
    new_list = n2[2]
  return (new_distance,new_status,new_list)

Elijo un nodo origen cualquiera de los que tengo

In [ ]:
ID_ORIGEN = 4826784

Defino una función para la obtención de los artículos de x grado del bfs (la llamo capa)

In [ ]:
def obtener_capa_del_bfs(adjlist, degree): 
    print(f"Capa {degree} del bfs")
    capa_degree = adjlist.filter(lambda x:x[1][0]==degree).map(lambda x:(x[0],1)).join(rdd_keys).map(lambda x:x[1][1])
    print(capa_degree.collect())

Primera pasada del bfs

In [ ]:
adjlist = adjlist.flatMap(lambda x:proc_node(x) if x[0]==ID_ORIGEN else [x])
adjlist = adjlist.reduceByKey(reduce_nodes)
capa_1 = adjlist.filter(lambda x:x[1][0]<999).map(lambda x:(x[0],1)).join(rdd_keys).map(lambda x:x[1][1])
print(f"Capa {1} del bfs")
print(capa_1.collect())

Capa 1 del bfs
['Kamatapur', 'Duars', 'Rabha', 'Ureilita', 'Assam', 'Mamoni Raisom Goswami', 'Anexo:Distritos del estado de Assam', 'Anexo:Distritos de la India por área y población', 'Anexo:Distritos de la India', 'Distrito de Goalpara']


Hago N pasadas del recorrido bfs e imprimo las capas

In [ ]:
for i in range(1,4):
  print(f"Nodo origen 'Bandera_cÃ³digo_del_COI_TPE'")
  adjlist = adjlist.flatMap(lambda x:proc_node(x) if x[1][1]=='P' else [x])
  adjlist = adjlist.reduceByKey(reduce_nodes)
  obtener_capa_del_bfs(adjlist, i)

Nodo origen 'Bandera_cÃ³digo_del_COI_TPE'
Capa 1 del bfs
['Kamatapur', 'Duars', 'Rabha', 'Ureilita', 'Assam', 'Mamoni Raisom Goswami', 'Anexo:Distritos del estado de Assam', 'Anexo:Distritos de la India por área y población', 'Anexo:Distritos de la India', 'Distrito de Goalpara']
Nodo origen 'Bandera_cÃ³digo_del_COI_TPE'
Capa 2 del bfs
['Tiberioides borealis', 'Dilipa morgiana', 'Tangla', 'Apagón de India de 2012', 'Fuerzas Armadas de India', 'Cyclommatus strigiceps assamensis', 'Bokajan', 'Pinalia amica', 'Violencia de diciembre de 2014 en Assam', 'Aderus arthriticus', 'Tinsukia', 'Stenothemus robustus', 'Pueblo bengalí', 'Sapatgram', 'Anabas testudineus', 'Wikiproyecto:Mujeres/Artículos faltantes/Biólogas', 'Karbi', 'Congea tomentosa', 'Sharchop', 'Imperio Maurya', 'English breakfast tea', 'Rucervus duvaucelii', 'Mahua Moitra', 'Polyura moori', 'Elecciones generales de India de 1977', 'Neoeucirrhichthys maydelli', 'Digboi', 'Eparquía de San Juan Crisóstomo de Gurgaon', 'Cymbidium mas